### 台指選擇權日內逐筆成交資訊

註:僅能抓近30天

In [1]:
from pandas.core.frame import DataFrame
from datetime import datetime
import pandas as pd
import requests
import io
import zipfile


def get_option_mtf(date: str) -> DataFrame:

    date = '_'.join([date[0:4], date[4:6], date[6:8]])
    
    url = 'https://www.taifex.com.tw/file/taifex'
    url += '/Dailydownload/OptionsDailydownloadCSV'
    url += f'/OptionsDaily_{date}.zip'
    res = requests.get(url)

    fractionSharesZip = io.BytesIO(res.content)
    fractionSharesZip = zipfile.ZipFile(fractionSharesZip)
    fractionSharesZip.extractall('')

    with open(f'OptionsDaily_{date}.csv', encoding='big5') as f:
        data = []
        for row in f:
            ele = []
            row = row.strip().replace('\t', '').split(',')
            for d in row:
                ele.append(d.strip())
            data.append(ele)

    df = pd.DataFrame(data[2:], columns=data[0])

    df['成交日期'] = df['成交日期'] + df['成交時間']
    df['成交日期'] = df['成交日期'].apply(lambda x: datetime.strptime(x, '%Y%m%d%H%M%S'))

    df.drop('成交時間', axis=1, inplace=True)
    df.drop('開盤集合競價', axis=1, inplace=True)

    df.columns = ['daytime', 'merchandise', 'strike', 'maturity', 'cp', 'price', 'volume']

    df['strike'] = df['strike'].astype(float)
    df['price'] = df['price'].astype(float)
    df['volume'] = df['volume'].astype(int)

    return df


In [2]:
get_option_mtf('20210601')

,daytime,merchandise,strike,maturity,cp,price,volume
0,2021-06-01 11:12:37,CCO,60.0,202107,C,1.97,3
1,2021-06-01 11:12:37,CCO,60.0,202107,C,1.97,3
2,2021-06-01 13:22:59,CDO,525.0,202107,P,4.16,1
3,2021-06-01 13:22:59,CDO,525.0,202107,P,4.16,1
4,2021-06-01 13:25:18,CDO,550.0,202106,P,1.58,1
...,...,...,...,...,...,...,...
388791,2021-05-31 22:06:02,TXO,20300.0,202107,C,4.00,2
388792,2021-06-01 09:59:57,TXO,20300.0,202107,C,4.20,1
388793,2021-06-01 09:59:57,TXO,20300.0,202107,C,4.20,1
388794,2021-06-01 10:03:05,TXO,20300.0,202107,C,4.40,2
